<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module4/M4_1/0_Integer_Furniture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integer Programming: Motivation (Furniture)
**OPIM 5641: Business Decision Modeling - University of Connecticut**

Related Readings:
* `Pyomo Cookbook`: https://github.com/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/02.01-Production-Models-with-Linear-Constraints.ipynb
* `Powell`: Chapter 11 (Integer Optimization)

## Background
One of the basic assumptions in linear programming models relates to divisibility of the decision variables: Fractional values of decision variables make sense either literally (12.6 tons of steel can be produced) or as a modeling approximation (in practical terms, 1,776.2 trucks is the same as 1,776). 

The optimal solution of a linear program may therefore contain fractional decision variables, and this is appropriate—or at least tolerable—in most applications. In some cases, however, it may be necessary to ensure that some or all of the decision variables take on integer values. Accommodating the requirement that variables must be integers is the subject of integer programming.

Consider the product mix for chairs, desks, and tables discussed in the Veerman Furniture model of Chapter 9.As it turned out,the optimal solution contains no chairs,275 desks, and 100 tables, so that the decision variables happen to all be integers.

## Let's tinker... update the constraints...
Suppose instead that the problem occurs with only 1800 fabrication hours available, rather than the 1,850 of the base case.


As you will see, **the optimal product mix includes 266.67 desks**. Is the fractional number of desks meaningful?

Let's explore what we can do as a first entry into integer programming.

## Setup Your Environment/Imports

In [ ]:
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

# Example: Furniture
*Section 9.2 (Powell) - Chairs, Desks and Tables*

**Problem Description:**
Veerman Furniture Company makes three kinds of ofﬁce furniture: chairs, desks, and tables. Each product requires some labor in the parts fabrication department, the assembly department, and the shipping department. The furniture is sold through a regional distributor, who has estimated the maximum potential sales for each product in the coming quarter. Finally,the accounting department has provided some data showing the proﬁt contributions on each product.The decision problem isto determine the product mix—that is, to maximize Veerman’s proﬁt for the quarter by choosing production quantities for the chairs,desks,and tables.

The following data summarizes the parameters of the problem:

Department | Chairs | Desks | Tables | Hours Available
--- | --- | --- | --- | ---
Fabrication | 4 | 6 | 2 | 1,800 (UPDATED)
Assembly | 3 | 5 | 7 | 2,400
Shipping | 3 | 2 | 4 | 1,500
--------------------------------------------------------------------------------
Demand Potential | 360 | 300 | 100 |
Profit (USD) | 15 | 24 | 18 |

**Define the Objective Function**

$Profit = 15C + 24D + 18T$

**Write the Constraints**

$Max(Z) = 15C + 24D + 18T$

subject to:
* $4C + 6D + 2T <= 1,800$ `updated!`
* $3C + 5D + 7T <= 2,400$
* $3C + 2D + 4T <= 1,500$ 
* $C        <= 360$ 
* $D      <=300$ 
* $T <=10$

Great! Now that your problem is defined - go code it up and solve it.

In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables
model.c = Var(domain=NonNegativeReals) # c for chairs
model.d = Var(domain=NonNegativeReals) # d for desks
model.t = Var(domain=NonNegativeReals) # t for tables

# declare objective
model.profit = Objective(
                      expr = 15*model.c + 24*model.d + 18*model.t, # values come from the table
                      sense = maximize)

# declare constraints
model.Constraint1 = Constraint(expr = 4*model.c + 6*model.d + 2*model.t <= 1800) # fabrication hours - updated!
model.Constraint2 = Constraint(expr = 3*model.c + 5*model.d + 7*model.t <= 2400) # assembly hours
model.Constraint3 = Constraint(expr = 3*model.c + 2*model.d + 4*model.t <= 1500) # shipping
model.Constraint4 = Constraint(expr = model.c <= 360) # c demand
model.Constraint5 = Constraint(expr = model.d <= 300) # d demand
model.Constraint6 = Constraint(expr = model.t <= 100) # t demand

In [ ]:
# show the model you've created
model.pprint()

3 Var Declarations
    c : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    d : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    t : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 15*c + 24*d + 18*t

6 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper  : Active
        None :  -Inf : 4*c + 6*d + 2*t : 1800.0 :   True
    Constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper  : Active
        None :  -Inf : 3*c + 

In [ ]:
# ensure you have cbc installed
# !apt-get install -y -qq coinor-cbc

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 8200.0
  Upper bound: 8200.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created s

In [ ]:
# show the results
print("Profit = ", model.profit(), " per week")
print("Chairs = ", model.c(), " units per week")
print("Desks = ", model.d(), " units per week")
print("Tables = ", model.t(), " units per week")

Profit =  8200.00008  per week
Chairs =  0.0  units per week
Desks =  266.66667  units per week
Tables =  100.0  units per week


# Options available to us
**WOAH!** 266.67 desks... what are we supposed to do with that?!

*   Just assume that your business does not run in such a perfect world, and that rounding error from one desk is fine - in fact, there is probably some uncertainty in your biz practices and this is bound to add a little uncertainty. You round up (or down) and move on.
*   Assume it is mission-critical that you have nice, integer outputs. This requires you to specify NonNegativeInteger values in your script (demonstrated below). 
  * This is a relatively boring implementation of integer programming - but it's our first step here.
  * The optimal solution (objective function) will always be the same or decrease - by forcing your model's decision variables to take on integer values, you effectively are forcing an 'integer constraint'!
  * Watch this video if you don't believe me (this professor has a lot of great videos, click through is playlist): https://www.youtube.com/watch?v=RhHhy-8sz-4



# Re-run the problem.
The only thing that will change is the incorporation of 'NonNegativeIntegers' when we declare our decision variables.

In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables
model.c = Var(domain=NonNegativeIntegers) # c for chairs
model.d = Var(domain=NonNegativeIntegers) # d for desks
model.t = Var(domain=NonNegativeIntegers) # t for tables

# declare objective
model.profit = Objective(
                      expr = 15*model.c + 24*model.d + 18*model.t, # values come from the table
                      sense = maximize)

# declare constraints
model.Constraint1 = Constraint(expr = 4*model.c + 6*model.d + 2*model.t <= 1800) # fabrication hours - updated!
model.Constraint2 = Constraint(expr = 3*model.c + 5*model.d + 7*model.t <= 2400) # assembly hours
model.Constraint3 = Constraint(expr = 3*model.c + 2*model.d + 4*model.t <= 1500) # shipping
model.Constraint4 = Constraint(expr = model.c <= 360) # c demand
model.Constraint5 = Constraint(expr = model.d <= 300) # d demand
model.Constraint6 = Constraint(expr = model.t <= 100) # t demand

In [ ]:
# show the model you've created
model.pprint()

3 Var Declarations
    c : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    d : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    t : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 15*c + 24*d + 18*t

6 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper  : Active
        None :  -Inf : 4*c + 6*d + 2*t : 1800.0 :   True
    Constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper  : Active
        None :  -Inf

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 8199.0
  Upper bound: 8199.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of binary variables: 0
  Number of integer variables: 3
  Number of nonzeros: 3
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
  

In [ ]:
# show the results
print("Profit = ", model.profit(), " per week")
print("Chairs = ", model.c(), " units per week")
print("Desks = ", model.d(), " units per week")
print("Tables = ", model.t(), " units per week")

Profit =  8199.0  per week
Chairs =  1.0  units per week
Desks =  266.0  units per week
Tables =  100.0  units per week


# Analysis
Our profit went down to 8199 from 8200 and now we are making a single chair. 

As we mentioned, imposing an integer constraint (even if just on the decision variables) will never improve your solution (think of it like red tape!). 

As a decision-maker, you may decide it doesn't make sense to manufacture a single chair. We'll talk more about the fixed and variable costs associated with manufacturing later on.